In [1]:
import os, openai
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv( r"C:\Users\Akash Giri\Documents\PyWork\PGDM_GenAI\myAPI.env"))

# openai_api_key  = os.getenv('OPENAI_KEY_PGDM') #('OPENAI_API_KEY')
# openai.api_key = openai_api_key

# from openai import OpenAI
# client = OpenAI(
#     api_key = os.environ["GEMINI_API_KEY"], 
#     base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
# )


True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse

from langchain.chat_models import init_chat_model

basic_model = init_chat_model(
    model = "gemini-2.0-flash",  #"claude-sonnet-4-5-20250929",
    api_key = os.environ["GEMINI_API_KEY"], 
    model_provider = "google_genai",
    temperature=0.5,
    timeout=10,
    max_tokens=100
)

# from langchain.chat_models import init_chat_model

advanced_model = init_chat_model(
    model = "gemini-2.5-flash",  #"claude-sonnet-4-5-20250929",
    api_key = os.environ["GEMINI_API_KEY"], 
    model_provider = "google_genai",
    temperature=0.5,
    timeout=10,
    max_tokens=100
)

# basic_model = client.chat.completions.create(
#     model = "gemini-2.0-flash", 
#     messages = [{"role": "user", "content": "Act as an assistant"}],  
#     # max_tokens = 
#     # temperature = 
#     # max_p =
#     # max_k =
# )

# advaced_model = client.chat.completions.create(
#     model = "gemini-2.0-flash", 
#     messages = [{"role": "user", "content": "Act as an assistant"}],  
#     # max_tokens = 
#     # temperature = 
#     # max_p =
#     # max_k =
# )

# basic_model = ChatOpenAI(model="gpt-4o-mini")
# advanced_model = ChatOpenAI(model="gpt-4o")

def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"
# 
tools=[get_weather]

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])

    if message_count > 10:
        # Use an advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    request.model = model
    return handler(request)

agent = create_agent(
    model=basic_model,  # Default model
    tools=tools,
    middleware=[dynamic_model_selection]
)

In [3]:
 agent.invoke({"messages": [{"role": "user", "content": "what is the weather at Delhi?"}]})

{'messages': [HumanMessage(content='what is the weather at Delhi?', additional_kwargs={}, response_metadata={}, id='9e4f98a7-a84e-4ed2-bc2f-d194e7d41b80'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "Delhi"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='lc_run--fef9b726-441b-4727-98d6-adc7c91291e8-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Delhi'}, 'id': 'eb865b43-8947-48c3-bb35-67a74357f662', 'type': 'tool_call'}], usage_metadata={'input_tokens': 21, 'output_tokens': 5, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="It's always sunny in Delhi!", name='get_weather', id='7830bdac-079a-48ff-a76c-9b431fbb6546', tool_call_id='eb865b43-8947-48c3-bb35-67a74357f662'),
  AIMessage(content="It's always sunny in Delhi!", additional_kwargs={}, response_meta

In [4]:
agent.invoke({"messages": [{"role": "user", "content": "what is the weather at Mumbai?"}]})

{'messages': [HumanMessage(content='what is the weather at Mumbai?', additional_kwargs={}, response_metadata={}, id='40010445-077c-4f52-800c-154462e4c32f'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "Mumbai"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='lc_run--2f4e5cbc-a03c-49dd-b68a-d729b8078e4c-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Mumbai'}, 'id': 'a2dc9a42-7580-4b7e-83bb-43e90e1e0a93', 'type': 'tool_call'}], usage_metadata={'input_tokens': 21, 'output_tokens': 5, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="It's always sunny in Mumbai!", name='get_weather', id='76f75dc6-3a79-411b-b445-3e2ad15df0f1', tool_call_id='a2dc9a42-7580-4b7e-83bb-43e90e1e0a93'),
  AIMessage(content="It's always sunny in Mumbai!", additional_kwargs={}, response

In [5]:
agent.stream({"messages": [{"role": "user", "content": "what is the weather at Delhi?"}]}, 
            stream_mode="values")

<generator object Pregel.stream at 0x0000021C589F06D0>

In [8]:
for event in agent:
    if "messages" in event:
        event["messages"][-1].pretty_print()

TypeError: 'CompiledStateGraph' object is not iterable